# Boost Switching regulator LM3478
- Supply: 15 V
- Output voltage: Between 20 and 40 V (theoretically up to 45 V)
	- Make the output voltage variable through some variable resistor (Manually / voltage controlled)

## Switching architecture
- Switching frequency: 100 kHz - 1 MHz
- PWM, current mode control architecture
- Only fits for low-side switching (no level shifting)

## Other features
- Thermal shutdown
- Short-cicuit protection
- Over-voltage protection
- **Internal soft-start**

# LTSpice Simulation
## Issue with simulation
- Vds gets to a max between 3 and 4 V (since Vcc is 3.3 V)
  - Vgs(th) is 2-4 V
  - It seems like the Vgs is simply somewhat too small for the fet to be in the linear conducting region

## Current mode control
### Operation
- ISEN-pin: Peak current is sensed through the MOSFET over the external resistor R4.
  - This forms a resistive divider with Rds_on and R4.
  - This voltage is fed into the +PWM_COMP of the PWM comparator
- FB-pin: Voltage fed back from the divider, fed into the  -ERR_AMP
- COMP: OUT_ERR_AMP
  - Added to the slope compensation ramp
  - Fed into -PWM_COMP

1. Device switching START:
  - Voltage > UVLO set point + FA/SD is pulled down
2. Switching cycle start
  - Oscillator sets RS-latch
  - DR pin is set too high -> MOSFET set high (peak current 1 amp)
3. At a certain point, +PWM_COMP > -PWM_COMP
  - RS latch will be reset
  - External Mosfet will turn off
4. Output capacitor C_out will discharge
  - V_out will reduce
  - Feedback voltage will be lower than the 1.25 V reference
  - Error amplifier will be positive
  - PWM_COMP_OUT will become more negative
  - RS latch will be set again by oscillator
5. Back to 2

### Overvoltage Protection
- FB: Additional input to +OVP_COMP
- Vfb + Vovp: input to -OVP_OMP
  - Vovp: 32-50 V
  - Vovp_hys is between 20 and 110 mV
- OV_COMP_OUT: Makes the FET output go low

So basically, if the feedback has an oscillation > 60 mV (ofn the 1.25 V reference), there will be a fet shutdown.


### Slope compensation ramp
- Normally CCM is inherently unstable at a duty cycle of 50 % or more.
  - In that case the inductor current diverges
  - So to reach a voltage of 40 volts we need a supply of at least 20 V

#### Slope compensation
Adding a resistor Rsl in the Isen-path.

However: higher slope compensation leads to less influence from the sensed current.

### Frequency adjust / shutdown
- External resistor at FA/SD is adapted to change the switching characteristics
  - R = 25 kOhm -> 1 MHz switching frequency
  - R = 200 kOhm -> 100 kHz switching frequency

#### Low vs High frequency
High frequency
- CON:
  - Higher inductor, dielectric, switching loss (Eddy currents + Hysteresis)
  - Your feedback loops bandwidth is higher, so more sensitive to parasitics and oscillations
- PRO:
  - smaller magnetics / caps

Reverse is true for lower frequencies

#### Shutdown pin
When applied voltage is > 1.35 V
- LM3478 stops switching

### Short-circuit Protection
- Isen > 343 mV -> short circuit current limit protection happens. (in case the MOSFET shorts out)



### Current-mode control
